In [99]:
from datetime import timedelta, datetime
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [100]:
data_path = "../data/1min/"
imputed_data_path = "../data/imputed/1min/"

try:
    os.mkdir("../data/imputed")
    os.mkdir(imputed_data_path)
except OSError as error:
    print(error)

stocks = [file.split('/')[-1][:-4] for file in os.listdir(data_path)]
datetime_format = "%Y-%m-%dT%X+03:00"

[Errno 17] File exists: '../data/imputed'


In [101]:
# read csv of SISE
historical_df = pd.read_csv(data_path + "AKBNK.csv")

In [102]:
historical_df.tail()

,Unnamed: 0,symbol,signal_time,date_time,close,high,low,open,total_quantity,volume,weighted_average
422999,32999,AKBNK,1609167360000,2020-12-28T17:56:00+03:00,6.55,6.55,6.54,6.54,457516.0,2.996671e+06,6.549872
423000,33000,AKBNK,1609167420000,2020-12-28T17:57:00+03:00,6.55,6.56,6.55,6.55,31862.0,2.087247e+05,6.550899
423001,33001,AKBNK,1609167480000,2020-12-28T17:58:00+03:00,6.55,6.56,6.55,6.55,329625.0,2.159175e+06,6.550399
423002,33002,AKBNK,1609167540000,2020-12-28T17:59:00+03:00,6.55,6.55,6.54,6.54,88293.0,5.781336e+05,6.547898
423003,33003,AKBNK,1609167840000,2020-12-28T18:04:00+03:00,6.54,6.54,6.54,6.54,1093251.0,7.149861e+06,6.540000


In [103]:
historical_array = historical_df.to_numpy()

print(historical_array.shape)

(423004, 11)


In [95]:
class StockImputer:

    def __init__(self, data):
        self.raw_data = data
        self.imputed_data = []
        self.imputed_data_as_array = None

        self.START_DATE = datetime.strptime(data[0][3], "%Y-%m-%dT%X+03:00")
        self.START_TIMESTAMP = data[0][2]
        self.END_DATE = datetime.strptime(data[-1][3], "%Y-%m-%dT%X+03:00")
        self.END_TIMESTAMP = data[-1][2]

        self.filler_index = 0
        self.parser_index = 0

        self.filler_row = self.raw_data[0].copy()
        self.parser_row = self.raw_data[0]

    def _add_row(self, row, keep_values=False):
        
        if not keep_values:
            self.filler_row[4:] = row[4:]  # take the values of current row
        
        self.imputed_data.append(self.filler_row.copy())  # add row to the imputed dataset

        self.filler_row[0] += 1  # inc index
        self.filler_row[2] += 60000  # inc timestamp

        # inc time string
        self.filler_row[3] = datetime.fromtimestamp(self.filler_row[2] / 1000).strftime("%Y-%m-%dT%X+03:00")

    def _add_until_row(self, row):
        
        while self.filler_row[2] != row[2]:
            self._add_row(row, keep_values=True)  # add rows between
        
        self._add_row(row, keep_values=False)  # add last row too
        
    def impute(self):

        for row in tqdm(self.raw_data):
            
            if self.filler_row[2] == row[2]:
                self._add_row(row)
                continue
            
            else:
                if (row[2] - self.filler_row[2]) / 60000 > 30:
                    # day or session changed
                    self.filler_row = row.copy()
                    self._add_row(row)
                    continue
                
                else:
                    self._add_until_row(row)
                    continue
        self.imputed_data_as_array = np.array(self.imputed_data)
        

In [104]:
imputer = StockImputer(historical_array[:500])

print(f"""
Start Date: {imputer.START_DATE.strftime(datetime_format)}
End Date: {imputer.END_DATE.strftime(datetime_format)}
""")


Start Date: 2017-01-02T10:00:00+03:00
End Date: 2017-01-03T14:48:00+03:00



In [105]:
imputer.impute()  # fill missing values

100%|██████████| 500/500 [00:00<00:00, 43181.48it/s]


In [110]:
not_imputed_df = pd.DataFrame(data=imputer.raw_data, columns=[i+"_RAW" for i in historical_df.columns])
imputed_df = pd.DataFrame(data=imputer.imputed_data_as_array, columns=[i+"_IMP" for i in historical_df.columns])

pd.concat([not_imputed_df[:25], imputed_df[:25]], axis = 1)[['date_time_RAW', 'close_RAW', 'date_time_IMP', 'close_IMP']]

,date_time_RAW,close_RAW,date_time_IMP,close_IMP
0,2017-01-02T10:00:00+03:00,6.243333,2017-01-02T10:00:00+03:00,6.243333
1,2017-01-02T10:01:00+03:00,6.251348,2017-01-02T10:01:00+03:00,6.251348
2,2017-01-02T10:02:00+03:00,6.243333,2017-01-02T10:02:00+03:00,6.243333
3,2017-01-02T10:03:00+03:00,6.243333,2017-01-02T10:03:00+03:00,6.243333
4,2017-01-02T10:04:00+03:00,6.251348,2017-01-02T10:04:00+03:00,6.251348
5,2017-01-02T10:05:00+03:00,6.251348,2017-01-02T10:05:00+03:00,6.251348
6,2017-01-02T10:06:00+03:00,6.259362,2017-01-02T10:06:00+03:00,6.259362
7,2017-01-02T10:07:00+03:00,6.251348,2017-01-02T10:07:00+03:00,6.251348
8,2017-01-02T10:08:00+03:00,6.251348,2017-01-02T10:08:00+03:00,6.251348
9,2017-01-02T10:09:00+03:00,6.235319,2017-01-02T10:09:00+03:00,6.235319


In [111]:
imputer_full = StockImputer(historical_array)

print(f"""
Start Date: {imputer_full.START_DATE.strftime(datetime_format)}
End Date: {imputer_full.END_DATE.strftime(datetime_format)}
""")

imputer_full.impute()

  3%|▎         | 11607/423004 [00:00<00:07, 57248.94it/s]


Start Date: 2017-01-02T10:00:00+03:00
End Date: 2020-12-28T18:04:00+03:00



100%|██████████| 423004/423004 [00:05<00:00, 79761.86it/s]


In [112]:
print(imputer_full.raw_data.shape)
print(imputer_full.imputed_data_as_array.shape)
ratio = (imputer_full.imputed_data_as_array.shape[0] / imputer_full.raw_data.shape[0] - 1) * 100
print(f"Missing values: {round(ratio, 2)}%")

(423004, 11)
(442228, 11)
Missing values: 4.54%


# Impute All Stocks

In [114]:
for stock in stocks:
    historical_df = pd.read_csv(data_path + stock + ".csv")
    historical_array = historical_df.to_numpy()
    
    imputer = StockImputer(historical_array)
    imputer.impute()
    
    imputed_df = pd.DataFrame(data=imputer.imputed_data_as_array, columns=historical_df.columns)
    
    imputed_df.to_csv(imputed_data_path + stock + "_full.csv")


100%|██████████| 359190/359190 [00:05<00:00, 61348.04it/s]
